In [15]:
# general
import numpy as np
import pandas as pd
from IPython.display import display
import importlib

# first used in exercise one
import kernelsvm as svm
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel
from sklearn.datasets import fetch_mldata
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [16]:
importlib.reload(svm)

<module 'kernelsvm' from '/Users/andrewenfield/work/github/Data558/Week09/kernelsvm.py'>

In [17]:
x_simple = np.array([3,2,0,1,-1,-2]).reshape(3,2)
x_simple

array([[ 3,  2],
       [ 0,  1],
       [-1, -2]])

Note: Per the request in the "Collaboration policy" note, I've discussed at least part of this assignment with many of the MS employees in the class, including Abhishek, Geoff, Suman, and Charles. (Different weeks/different assignments have different people, depending upon who attends our study groups, but I'll probably just include this blurb w/ each homework since it's generally correct.) I've also gotten input from the discussion board.

# Exercise one

_Compute the gradient ∇F(α) of F.

_Write a function computegram that computes, for any set of datapoints x1, . . . , xn, the kernel matrix K._

I implemented the computegram_linear function, and all of the supporting functions including the gradient and objective functions, in the file kernelsvm.py, which I imported into this notebook with the alias svm. I also include my (relatively minimal) unit tests, in linearsvm-test.py.

In [18]:
svm.computegram_linear(x_simple)

array([[13,  2, -7],
       [ 2,  1, -2],
       [-7, -2,  5]])

In [19]:
svm.computegram_polynomial(x_simple, 2, 1)

array([[196,   9,  36],
       [  9,   4,   1],
       [ 36,   1,  36]])

_Write a function kerneleval that computes, for any set of datapoints x1, . . . , xn and a new datapoint x⋆, the vector of kernel evaluations [k(x1, x⋆), . . . , k(xn, x⋆)]T._

My understanding here is that I'm not using the kernel matrix - that is, the reference to 'kernel evaluation' means to use the kernel _function_, not the kernel matrix. I think this is just (part of - since no alpha weighting yet) the predict step, which - for a linear kernel - is the sum of the inner products between each observation and the value to be predicted, weighted by each learned value of alpha.

In [20]:
svm.kerneleval_linear(x_simple, np.array([2,2]))

array([10,  2, -6])

**TODO** Is there a way this function should be generalized to work with different kinds of kernels?

_Consider the MNIST dataset. You can find instructions on how to download it here: http://scikit-learn.org/stable/datasets/mldata.html. Pick two classes of your choice. Your are going to work on the dataset consisting of the data from these two classes. Standardize the data, if you have not done so already._

In [21]:
mnist = fetch_mldata('MNIST original')
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

We'll use 1 and 8. 

In [22]:
ones_and_eights = (mnist.target == 1) | (mnist.target == 8)
X = mnist.data[ones_and_eights]
y = mnist.target[ones_and_eights]
X.shape, y.shape, np.unique(y)

((14702, 784), (14702,), array([ 1.,  8.]))

In [23]:
X_scaled = preprocessing.scale(X)
X_scaled.shape

/Users/andrewenfield/miniconda3/envs/data558/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


(14702, 784)

In [24]:
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X_scaled, y)
X_scaled_train.shape, X_scaled_test.shape, y_train.shape, y_test.shape

((11026, 784), (3676, 784), (11026,), (3676,))

_Write a function mysvm that implements the fast gradient algorithm to train the kernel support vector machine with the squared hinge loss. The function takes as input the initial step-size value for the backtracking rule and a maximum number of iterations._

As noted above, I implemented this in the kernelsvm.py file. Instead of creating a function named mysvm, I just call my fastgradalgo function and pass in references to the kernel SVM gradient and objective functions (which are also implemented in the same file).

_Train your kernel support vector machine with the squared hinge loss and the polyno- mial kernel of order 7 on the the MNIST dataset, tuning the regularization parameter λ using cross-validation._

In [25]:
importlib.reload(svm)

<module 'kernelsvm' from '/Users/andrewenfield/work/github/Data558/Week09/kernelsvm.py'>

In [26]:
t_init = 0.01
max_iters = 10

In [29]:
results = svm.fastgradalgo(
    svm.computegram_linear(X_scaled_train), y_train, t_init=t_init, 
    grad_func = svm.compute_kernelsvm_gradient, 
    obj_func = svm.compute_kernelsvm_objective, 
    lam=1, max_iter=max_iters, t_func=svm.backtracking)
results[-3:]

,0,1,2,3,4,5,6,7,8,9,...,11016,11017,11018,11019,11020,11021,11022,11023,11024,11025
8,-3.919580e-08,-1.673196e-08,3.705885e-07,3.850453e-08,-9.724808e-08,1.610068e-07,-1.027588e-08,4.503866e-08,-2.030279e-08,-4.697711e-09,...,-1.730627e-08,-3.084553e-09,2.561413e-08,-1.065267e-08,-1.421242e-08,1.336365e-07,-1.945003e-09,-1.063102e-08,-1.267999e-08,6.183394e-09
9,-3.948997e-08,-1.682551e-08,3.727553e-07,3.877792e-08,-9.785205e-08,1.618558e-07,-1.033496e-08,4.520049e-08,-2.041350e-08,-4.729573e-09,...,-1.740627e-08,-3.107938e-09,2.572290e-08,-1.071159e-08,-1.429004e-08,1.342805e-07,-2.057912e-09,-1.069192e-08,-1.275327e-08,6.205659e-09
10,-3.970392e-08,-1.689355e-08,3.743312e-07,3.897675e-08,-9.829130e-08,1.624732e-07,-1.037793e-08,4.531819e-08,-2.049401e-08,-4.752746e-09,...,-1.747899e-08,-3.124945e-09,2.580201e-08,-1.075445e-08,-1.434650e-08,1.347489e-07,-2.140028e-09,-1.073621e-08,-1.280656e-08,6.221851e-09


In [44]:
results[-1:][80].values

array([ -2.63744307e-08])

_Compare the performance of kernel SVMs with different kernels (polynomial kernels with different orders, Gaussian RBF with different bandwidths, etc.)._